In [178]:
import arcpy
import os
import math
import datetime
import uuid

In [179]:
def find_field_by_pattern(fields, pattern_list):
    """Helper function to find a field based on patterns"""
    for field in fields:
        field_name = field.name.upper()
        if all(pattern in field_name for pattern in pattern_list):
            return field.name
    return None

def find_exact_field(fields, name):
    """Helper function to find a field based on exact name"""
    for field in fields:
        if field.name.upper() == name.upper():
            return field.name
    return None

In [180]:
# Ask user for threshold values
print("====== PCI Difference Threshold Input ======")
lower_threshold = float(input("Enter LOWER threshold value (include points with PCI difference <= this value): "))
higher_threshold = float(input("Enter HIGHER threshold value (include points with PCI difference >= this value): "))
print(f"Will include points with PCI difference <= {lower_threshold} OR >= {higher_threshold}")

# New input for work history filtering
work_history_filter = input("Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): ")
work_history_filter = work_history_filter.lower().strip()
if work_history_filter == 'yes':
    print("Will only include rows above the positive threshold that have NO work history (empty MRTreatmentName)")
else:
    print("Will include all rows that meet threshold criteria regardless of work history")
print("============================================")

====== PCI Difference Threshold Input ======
Enter LOWER threshold value (include points with PCI difference <= this value): -10
Enter HIGHER threshold value (include points with PCI difference >= this value): 15
Will include points with PCI difference <= -10.0 OR >= 15.0
Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): Yes
Will only include rows above the positive threshold that have NO work history (empty MRTreatmentName)


In [181]:
# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")
print(f"Current project: {aprx.filePath}")

# Get the active map and find the WildomarPMPJordan feature layer
PMP_Path = None
active_map = None

# Search for the feature class in all maps
for m in aprx.listMaps():
    for lyr in m.listLayers():
        if lyr.isFeatureLayer and "SSstreet" in lyr.name:
            PMP_Path = lyr.dataSource
            wildomar_layer = lyr
            active_map = m
            print(f"Found layer: {lyr.name} in map: {m.name}")
            break
    if PMP_Path:
        break

if not PMP_Path:
    print("ERROR: WildomarPMPJordan feature class not found in the project.")
    # raise RuntimeError("WildomarPMPJordan feature class not found in the project.")

# Define WGS84 (lat/long) spatial reference for external API URLs
wgs84 = arcpy.SpatialReference(4326)  # EPSG code for WGS84

# Get the spatial reference of the feature layer
desc = arcpy.Describe(wildomar_layer)
layer_sr = desc.spatialReference

print(f"Feature layer spatial reference: {layer_sr.name} ({layer_sr.factoryCode})")
print(f"Target spatial reference: WGS84 (4326)")


Current project: C:\Users\JLin\Downloads\QC Tool\QC Tool.aprx
Found layer: SSstreet in map: Map
Feature layer spatial reference: NAD_1983_StatePlane_California_VI_FIPS_0406_Feet (2230)
Target spatial reference: WGS84 (4326)


In [182]:
# Find the PCI Differences table
pci_table = None
for table in active_map.listTables():
    if "PCI Differences" in table.name:
        pci_table = table
        print(f"Found standalone table: {table.name}")
        break

if not pci_table:
    print("WARNING: PCI Differences table not found in the active map.")
    # Use a raise statement here if you want to stop execution

Found standalone table: PCI Differences


In [183]:
# Determine field names in the feature layer
print("\nLooking for key fields in the feature layer...")
shapefile_fields = arcpy.ListFields(wildomar_layer)

# Use exact field names from your data
street_id_field = find_exact_field(shapefile_fields, "Street_ID")
section_id_field = find_exact_field(shapefile_fields, "Section_ID")
street_name_field = find_exact_field(shapefile_fields, "Street_Name")
begin_loc_field = find_exact_field(shapefile_fields, "Begin_Location")
end_loc_field = find_exact_field(shapefile_fields, "End_Location")

# Print found fields with clear confirmation
if street_id_field:
    print(f"Found street ID field: {street_id_field}")
else:
    print("WARNING: Street_ID field not found in feature layer")
    
if section_id_field:
    print(f"Found section ID field: {section_id_field}")
else:
    print("WARNING: Section_ID field not found in feature layer")
    
if street_name_field:
    print(f"Found street name field: {street_name_field}")
else:
    print("WARNING: Street_Name field not found in feature layer")
    
if begin_loc_field:
    print(f"Found begin location field: {begin_loc_field}")
else:
    print("WARNING: Begin_Location field not found in feature layer")
    
if end_loc_field:
    print(f"Found end location field: {end_loc_field}")
else:
    print("WARNING: End_Location field not found in feature layer")


Looking for key fields in the feature layer...


In [184]:
# Determine field names in the PCI table
print("\nLooking for key fields in the PCI Differences table...")
pci_fields = arcpy.ListFields(pci_table)

# Use exact field names from your screenshot
pci_street_id_field = find_exact_field(pci_fields, "Street_ID")
pci_section_id_field = find_exact_field(pci_fields, "Section_ID")
prev_insp_pci_field = find_exact_field(pci_fields, "Prev_Insp_PCI")
last_insp_pci_field = find_exact_field(pci_fields, "Last_Insp_PCI")
prev_insp_date_field = find_exact_field(pci_fields, "Prev_Insp_Date")
m_r_date_field = find_exact_field(pci_fields, "M_R_Date")
m_r_treatment_field = find_exact_field(pci_fields, "M_R_Treatment_Name")
m_r_pci_field = find_exact_field(pci_fields, "M_R_PCI")
last_insp_date_field = find_exact_field(pci_fields, "Last_Insp_Date")
begin_loc_field_pci = find_exact_field(pci_fields, "Begin_Location")
end_loc_field_pci = find_exact_field(pci_fields, "End_Location")

# Print status of each field
for field_name, field_var in [
    ("Street_ID", pci_street_id_field),
    ("Section_ID", pci_section_id_field),
    ("Prev_Insp_PCI", prev_insp_pci_field),
    ("Last_Insp_PCI", last_insp_pci_field),
    ("Prev_Insp_Date", prev_insp_date_field),
    ("M_R_Date", m_r_date_field),
    ("M_R_Treatment_Name", m_r_treatment_field),
    ("M_R_PCI", m_r_pci_field),
    ("Last_Insp_Date", last_insp_date_field),
    ("Begin_Location", begin_loc_field_pci),
    ("End_Location", end_loc_field_pci)
]:
    if field_var:
        print(f"Found field: {field_name} = {field_var}")
    else:
        print(f"WARNING: {field_name} field not found in PCI table")

# Check if we have the necessary fields
if not pci_street_id_field or not pci_section_id_field:
    print("ERROR: Could not find Street ID and Section ID fields in the PCI table.")
    # Use a raise statement here if you want to stop execution

if not street_id_field and not section_id_field:
    print("ERROR: Could not find Street ID or Section ID fields in the feature layer.")
    # Use a raise statement here if you want to stop execution

# Check if PCI fields were found in the table
if not prev_insp_pci_field or not last_insp_pci_field:
    print("ERROR: Could not find Prev_Insp_PCI or Last_Insp_PCI fields in the PCI table.")
    # Use a raise statement here if you want to stop execution
else:
    print(f"Found both PCI fields in the table. Will calculate difference: {prev_insp_pci_field} - {last_insp_pci_field}")


Looking for key fields in the PCI Differences table...
Found field: Street_ID = Street_ID
Found field: Section_ID = Section_ID
Found field: Prev_Insp_PCI = Prev_Insp_PCI
Found field: Last_Insp_PCI = Last_Insp_PCI
Found field: Prev_Insp_Date = Prev_Insp_Date
Found field: M_R_Date = M_R_Date
Found field: M_R_Treatment_Name = M_R_Treatment_Name
Found field: M_R_PCI = M_R_PCI
Found field: Last_Insp_Date = Last_Insp_Date
Found field: Begin_Location = Begin_Location
Found field: End_Location = End_Location
ERROR: Could not find Street ID or Section ID fields in the feature layer.
Found both PCI fields in the table. Will calculate difference: Prev_Insp_PCI - Last_Insp_PCI


In [185]:
# Create a dictionary to store PCI data for quicker lookups
print("\nReading PCI Differences data...")

# Debug: check a few records first
print("\nExamining PCI data types and values...")
sample_count = 0
sample_fields = [pci_street_id_field, pci_section_id_field]
if prev_insp_pci_field:
    sample_fields.append(prev_insp_pci_field)
if last_insp_pci_field:
    sample_fields.append(last_insp_pci_field)

with arcpy.da.SearchCursor(pci_table, sample_fields) as cursor:
    for row in cursor:
        street_id = str(row[0]).strip() if row[0] else "None"
        section_id = str(row[1]).strip() if row[1] else "None"
        
        print(f"Sample {sample_count}: {street_id}-{section_id}")
        field_idx = 2
        if prev_insp_pci_field:
            prev_pci = row[field_idx]
            print(f"  Prev PCI: {prev_pci} (type: {type(prev_pci)})")
            field_idx += 1
        if last_insp_pci_field:
            last_pci = row[field_idx]
            print(f"  Last PCI: {last_pci} (type: {type(last_pci)})")
        
        sample_count += 1
        if sample_count >= 5:  # Just show a few samples
            break

pci_data = {}

# Determine which fields to fetch from the PCI table
pci_fields_to_fetch = [pci_street_id_field, pci_section_id_field]

# Add required fields if they exist
if prev_insp_pci_field:
    pci_fields_to_fetch.append(prev_insp_pci_field)
if last_insp_pci_field:
    pci_fields_to_fetch.append(last_insp_pci_field)
if prev_insp_date_field:
    pci_fields_to_fetch.append(prev_insp_date_field)
if m_r_date_field:
    pci_fields_to_fetch.append(m_r_date_field)
if m_r_treatment_field:
    pci_fields_to_fetch.append(m_r_treatment_field)
if m_r_pci_field:
    pci_fields_to_fetch.append(m_r_pci_field)
if last_insp_date_field:
    pci_fields_to_fetch.append(last_insp_date_field)
# Add begin/end location fields from PCI table if they exist
if begin_loc_field_pci:
    pci_fields_to_fetch.append(begin_loc_field_pci)
if end_loc_field_pci:
    pci_fields_to_fetch.append(end_loc_field_pci)

with arcpy.da.SearchCursor(pci_table, pci_fields_to_fetch) as cursor:
    for row in cursor:
        street_id = str(row[0]).strip() if row[0] else ""
        section_id = str(row[1]).strip() if row[1] else ""
        
        # Create key in "Street_ID - Section_ID" format
        key = f"{street_id} - {section_id}"
        
        record_data = {
            'street_id': street_id,
            'section_id': section_id
        }
        
        # Get field index
        field_idx = 2
        
        # Get PCI values if fields are available
        if prev_insp_pci_field:
            record_data['prev_pci'] = row[field_idx]
            field_idx += 1
        if last_insp_pci_field:
            record_data['last_pci'] = row[field_idx]
            field_idx += 1
        
        # Get date and treatment fields
        if prev_insp_date_field:
            record_data['prev_insp_date'] = row[field_idx]
            field_idx += 1
        if m_r_date_field:
            record_data['m_r_date'] = row[field_idx]
            field_idx += 1
        if m_r_treatment_field:
            record_data['m_r_treatment'] = row[field_idx]
            field_idx += 1
        if m_r_pci_field:
            record_data['m_r_pci'] = row[field_idx]
            field_idx += 1
        if last_insp_date_field:
            record_data['last_insp_date'] = row[field_idx]
            field_idx += 1
        
        # Get begin/end location fields from PCI table if they exist
        if begin_loc_field_pci:
            record_data['begin_loc_pci'] = row[field_idx]
            field_idx += 1
        if end_loc_field_pci:
            record_data['end_loc_pci'] = row[field_idx]
            field_idx += 1
        
        # Calculate PCI difference if both PCI fields are available
        if 'prev_pci' in record_data and 'last_pci' in record_data:
            prev_pci = record_data['prev_pci']
            last_pci = record_data['last_pci']
            
            # Check for valid numeric values before calculating
            if prev_pci is not None and last_pci is not None:
                try:
                    # Try both calculation methods
                    pci_diff_calc = float(prev_pci) - float(last_pci)
                    record_data['pci_diff_calc'] = pci_diff_calc
                    
                    # Check if the record meets threshold criteria
                    meets_threshold = False
                    
                    # For lower threshold, we include regardless of work history
                    if pci_diff_calc <= lower_threshold:
                        meets_threshold = True
                        print(f"Record {key} included: PCI difference {pci_diff_calc} <= {lower_threshold}")
                    
                    # For higher threshold, we may filter based on work history
                    elif pci_diff_calc >= higher_threshold:
                        if work_history_filter.lower().strip() == 'yes':
                            # Check if there is NO work history (empty treatment field)
                            has_work_history = 'm_r_treatment' in record_data and record_data['m_r_treatment']
                            if not has_work_history:
                                meets_threshold = True
                                print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold} with NO work history")
                            else:
                                print(f"Record {key} excluded: PCI difference {pci_diff_calc} >= {higher_threshold} but HAS work history")
                        else:
                            # Include all records above higher threshold
                            meets_threshold = True
                            print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold}")
                    else:
                        print(f"Record {key} excluded: PCI difference {pci_diff_calc} not within thresholds")
                    
                    # Store record if it meets criteria
                    if meets_threshold:
                        record_data['meets_criteria'] = True
                        pci_data[key] = record_data
                    else:
                        record_data['meets_criteria'] = False
                        pci_data[key] = record_data
                        
                except (ValueError, TypeError):
                    print(f"Warning: Could not calculate PCI difference for {key}. Values: {prev_pci}, {last_pci}")

print(f"Loaded {len([k for k, v in pci_data.items() if v.get('meets_criteria', True)])} records from PCI Differences table that meet threshold criteria.")


Reading PCI Differences data...

Examining PCI data types and values...
Sample 0: 100-1076
  Prev PCI: 74 (type: <class 'int'>)
  Last PCI: 60 (type: <class 'int'>)
Sample 1: 100-3084
  Prev PCI: 72 (type: <class 'int'>)
  Last PCI: 73 (type: <class 'int'>)
Sample 2: 100-3229
  Prev PCI: 77 (type: <class 'int'>)
  Last PCI: 69 (type: <class 'int'>)
Sample 3: 100-3232
  Prev PCI: 60 (type: <class 'int'>)
  Last PCI: 75 (type: <class 'int'>)
Sample 4: 100-3234
  Prev PCI: 78 (type: <class 'int'>)
  Last PCI: 59 (type: <class 'int'>)
Record 100 - 1076 excluded: PCI difference 14.0 not within thresholds
Record 100 - 3084 excluded: PCI difference -1.0 not within thresholds
Record 100 - 3229 excluded: PCI difference 8.0 not within thresholds
Record 100 - 3232 included: PCI difference -15.0 <= -10.0
Record 100 - 3234 included: PCI difference 19.0 >= 15.0 with NO work history
Record 100 - 7032 included: PCI difference 17.0 >= 15.0 with NO work history
Record 100 - 7448 excluded: PCI differenc

Record 1076 - 2769 included: PCI difference -13.0 <= -10.0
Record 1078 - 4910 excluded: PCI difference -6.0 not within thresholds
Record 1079 - 5620 excluded: PCI difference -4.0 not within thresholds
Record 1080 - 1799 excluded: PCI difference -5.0 not within thresholds
Record 1082 - 4046 excluded: PCI difference -4.0 not within thresholds
Record 1084 - 2298 excluded: PCI difference -1.0 not within thresholds
Record 1086 - 2301 excluded: PCI difference -1.0 not within thresholds
Record 1088 - 1340 included: PCI difference -24.0 <= -10.0
Record 1090 - 2048 included: PCI difference -15.0 <= -10.0
Record 1092 - 4794 excluded: PCI difference 0.0 not within thresholds
Record 1092 - 7124 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 1631 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 2606 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 3412 excluded: PCI difference -3.0 not within thresholds
Record 1096 - 5056 excluded: PCI difference

Record 1168 - 1513 excluded: PCI difference -1.0 not within thresholds
Record 1168 - 6992 excluded: PCI difference -1.0 not within thresholds
Record 1170 - 1412 included: PCI difference 20.0 >= 15.0 with NO work history
Record 1170 - 1415 included: PCI difference 46.0 >= 15.0 with NO work history
Record 1170 - 4278 excluded: PCI difference 4.0 not within thresholds
Record 1170 - 4279 excluded: PCI difference 13.0 not within thresholds
Record 1170 - 4280 included: PCI difference -11.0 <= -10.0
Record 1170 - 4281 excluded: PCI difference -3.0 not within thresholds
Record 1172 - 4514 included: PCI difference -71.0 <= -10.0
Record 1174 - 7482 included: PCI difference -23.0 <= -10.0
Record 1176 - 3497 included: PCI difference -28.0 <= -10.0
Record 1176 - 3503 included: PCI difference -21.0 <= -10.0
Record 1177 - 6499 excluded: PCI difference -3.0 not within thresholds
Record 1177 - 7526 excluded: PCI difference -2.0 not within thresholds
Record 1178 - 3351 included: PCI difference -49.0 <= 

Record 1284 - 3989 excluded: PCI difference -5.0 not within thresholds
Record 1284 - 4796 excluded: PCI difference -4.0 not within thresholds
Record 1284 - 4799 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 4800 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 5572 excluded: PCI difference -5.0 not within thresholds
Record 1284 - 5573 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 5574 excluded: PCI difference -5.0 not within thresholds
Record 1286 - 5575 excluded: PCI difference -4.0 not within thresholds
Record 1288 - 3961 excluded: PCI difference -3.0 not within thresholds
Record 1290 - 2958 excluded: PCI difference -2.0 not within thresholds
Record 1290 - 7077 excluded: PCI difference -2.0 not within thresholds
Record 1292 - 3462 excluded: PCI difference -4.0 not within thresholds
Record 1292 - 4205 excluded: PCI difference -4.0 not within thresholds
Record 1292 - 7076 excluded: PCI difference -2.0 not within thresholds
Record

Record 135 - 34431 included: PCI difference -10.0 <= -10.0
Record 135 - 4015 included: PCI difference -10.0 <= -10.0
Record 135 - 4037 excluded: PCI difference 1.0 not within thresholds
Record 135 - 4103 excluded: PCI difference 2.0 not within thresholds
Record 135 - 4104 excluded: PCI difference 1.0 not within thresholds
Record 135 - 4177 included: PCI difference -10.0 <= -10.0
Record 135 - 4244 included: PCI difference -17.0 <= -10.0
Record 135 - 5272 excluded: PCI difference 1.0 not within thresholds
Record 135 - 5301 excluded: PCI difference -4.0 not within thresholds
Record 135 - 5319 excluded: PCI difference -2.0 not within thresholds
Record 135 - 5320 excluded: PCI difference -8.0 not within thresholds
Record 135 - 5345 included: PCI difference -11.0 <= -10.0
Record 135 - 5989 excluded: PCI difference -4.0 not within thresholds
Record 1350 - 5852 excluded: PCI difference -9.0 not within thresholds
Record 1352 - 7218 excluded: PCI difference -3.0 not within thresholds
Record 1354

...

Record 1766 - 5678 included: PCI difference -69.0 <= -10.0
Record 1766 - 5679 included: PCI difference -46.0 <= -10.0
Record 1766 - 5680 included: PCI difference -62.0 <= -10.0
Record 1768 - 1818 excluded: PCI difference -4.0 not within thresholds
Record 1770 - 2418 excluded: PCI difference -4.0 not within thresholds
Record 1770 - 2941 included: PCI difference -16.0 <= -10.0
Record 1772 - 2125 excluded: PCI difference -5.0 not within thresholds
Record 1774 - 7217 excluded: PCI difference -5.0 not within thresholds
Record 1776 - 2026 excluded: PCI difference -6.0 not within thresholds
Record 1776 - 6301 excluded: PCI difference -4.0 not within thresholds
Record 1776 - 6302 excluded: PCI difference -3.0 not within thresholds
Record 1776 - 6303 excluded: PCI difference -2.0 not within thresholds
Record 1778 - 4528 excluded: PCI difference 4.0 not within thresholds
Record 1778 - 5951 excluded: PCI difference 2.0 not within thresholds
Record 1778 - 5952 excluded: PCI difference 12.0 not wit

Record 3628 - 1432 excluded: PCI difference 29.0 >= 15.0 but HAS work history
Record 3630 - 3311 excluded: PCI difference 10.0 not within thresholds
Record 3632 - 3309 included: PCI difference 15.0 >= 15.0 with NO work history
Record 3634 - 6069 included: PCI difference -12.0 <= -10.0
Record 3634 - 6296 included: PCI difference -15.0 <= -10.0
Record 3636 - 1122 included: PCI difference -11.0 <= -10.0
Record 3638 - 3922 included: PCI difference -11.0 <= -10.0
Record 3640 - 1485 included: PCI difference 15.0 >= 15.0 with NO work history
Record 3640 - 1890 excluded: PCI difference -2.0 not within thresholds
Record 3642 - 1213 excluded: PCI difference -6.0 not within thresholds
Record 3644 - 1919 included: PCI difference -16.0 <= -10.0
Record 3646 - 1621 included: PCI difference 42.0 >= 15.0 with NO work history
Record 3648 - 4912 included: PCI difference -15.0 <= -10.0
Record 365 - 1055 excluded: PCI difference 4.0 not within thresholds
Record 365 - 3947 included: PCI difference -12.0 <= 

Record 375 - 2735 excluded: PCI difference 6.0 not within thresholds
Record 375 - 2757 excluded: PCI difference 9.0 not within thresholds
Record 375 - 2815 excluded: PCI difference -1.0 not within thresholds
Record 375 - 3062 excluded: PCI difference -1.0 not within thresholds
Record 375 - 3110 excluded: PCI difference -3.0 not within thresholds
Record 375 - 3312 included: PCI difference -18.0 <= -10.0
Record 375 - 3313 included: PCI difference -22.0 <= -10.0
Record 375 - 3341 excluded: PCI difference -7.0 not within thresholds
Record 375 - 3476 excluded: PCI difference 6.0 not within thresholds
Record 375 - 3559 excluded: PCI difference 13.0 not within thresholds
Record 375 - 3560 excluded: PCI difference -5.0 not within thresholds
Record 375 - 3658 excluded: PCI difference 10.0 not within thresholds
Record 375 - 3803 excluded: PCI difference 5.0 not within thresholds
Record 375 - 4006 excluded: PCI difference 12.0 not within thresholds
Record 375 - 4007 excluded: PCI difference -2.0 

Record 3804 - 6849 excluded: PCI difference -5.0 not within thresholds
Record 3806 - 4817 excluded: PCI difference -7.0 not within thresholds
Record 3808 - 6148 excluded: PCI difference -6.0 not within thresholds
Record 3810 - 6227 included: PCI difference -12.0 <= -10.0
Record 3812 - 4750 included: PCI difference -32.0 <= -10.0
Record 3812 - 7319 excluded: PCI difference -9.0 not within thresholds
Record 3814 - 1188 included: PCI difference -12.0 <= -10.0
Record 3816 - 3667 excluded: PCI difference 0.0 not within thresholds
Record 3818 - 4294 excluded: PCI difference 8.0 not within thresholds
Record 3820 - 3660 excluded: PCI difference 0.0 not within thresholds
Record 3822 - 5370 included: PCI difference 15.0 >= 15.0 with NO work history
Record 3822 - 5371 excluded: PCI difference 14.0 not within thresholds
Record 3824 - 2966 included: PCI difference -14.0 <= -10.0
Record 3826 - 1334 excluded: PCI difference -5.0 not within thresholds
Record 3828 - 2041 excluded: PCI difference -4.0 n

Record 3928 - 7477 excluded: PCI difference -4.0 not within thresholds
Record 3930 - 1517 excluded: PCI difference -3.0 not within thresholds
Record 3930 - 1553 excluded: PCI difference 12.0 not within thresholds
Record 3930 - 2365 included: PCI difference -12.0 <= -10.0
Record 3930 - 2952 excluded: PCI difference 2.0 not within thresholds
Record 3932 - 1084 included: PCI difference 16.0 >= 15.0 with NO work history
Record 3934 - 1831 excluded: PCI difference 7.0 not within thresholds
Record 3934 - 3452 excluded: PCI difference -3.0 not within thresholds
Record 3934 - 3453 included: PCI difference 17.0 >= 15.0 with NO work history
Record 3934 - 3454 excluded: PCI difference 4.0 not within thresholds
Record 3934 - 3455 excluded: PCI difference 3.0 not within thresholds
Record 3934 - 3456 included: PCI difference -10.0 <= -10.0
Record 3936 - 4293 included: PCI difference 20.0 >= 15.0 with NO work history
Record 3938 - 1746 included: PCI difference 18.0 >= 15.0 with NO work history
Record

In [186]:
# Outputs in default GDB
default_gdb = aprx.defaultGeodatabase
print(f"\nCreating outputs in default geodatabase: {default_gdb}")

# Create more user-friendly names
current_date = datetime.datetime.now().strftime("%Y%m%d")
formatted_thresholds = f"PCI_Diff_LE_{lower_threshold}_GE_{higher_threshold}"

# More recognizable names for outputs
table_name = f"QC_Table"
fc_name = f"QC_Points"
joined_fc_name = f"QC_Joined_Shapefile"

table_path = os.path.join(default_gdb, table_name)
fc_path = os.path.join(default_gdb, fc_name)
joined_fc_path = os.path.join(default_gdb, joined_fc_name)

# Thorough cleanup of ALL existing outputs with similar names to avoid duplicates
print("\nPerforming thorough cleanup of existing layers and tables...")

# Start by removing any matching layers from ALL maps in the project
for m in aprx.listMaps():
    print(f"Checking map: {m.name}")
    
    # Get lists of layers and tables to remove (collect first, remove after)
    layers_to_remove = []
    tables_to_remove = []
    
    # Check all layers
    for lyr in m.listLayers():
        try:
            if any(pattern in lyr.name for pattern in ["PCI_Threshold_Points", "PCI_Joined", "Midpoints", "TEMP_Midpoints"]):
                layers_to_remove.append(lyr)
                print(f"Will remove layer: {lyr.name}")
        except AttributeError:
            # Handle layers without valid name attribute
            layers_to_remove.append(lyr)
            print("Found a layer with no valid name attribute - will try to remove")
    
    # Check all tables
    for tbl in m.listTables():
        try:
            if "PCI_Threshold_Table" in tbl.name:
                tables_to_remove.append(tbl)
                print(f"Will remove table: {tbl.name}")
        except AttributeError:
            # Handle tables without valid name attribute
            tables_to_remove.append(tbl)
            print("Found a table with no valid name attribute - will try to remove")
    
    # Remove all identified layers
    for lyr in layers_to_remove:
        try:
            m.removeLayer(lyr)
            print(f"Removed layer from map")
        except Exception as e:
            print(f"Warning: Could not remove layer. Error: {str(e)}")
    
    # Remove all identified tables
    for tbl in tables_to_remove:
        try:
            m.removeTable(tbl)
            print(f"Removed table from map")
        except Exception as e:
            print(f"Warning: Could not remove table. Error: {str(e)}")

# Delete ALL matching feature classes and tables in the geodatabase
arcpy_workspace = aprx.defaultGeodatabase
arcpy.env.workspace = arcpy_workspace

# Define patterns to match ALL output types
pattern_list = [
    "*PCI_Threshold_Points*", 
    "*PCI_Threshold_Table*", 
    "*Midpoints*", 
    "*TEMP_Midpoints*",
    "*PCI_Joined*"
]

# Delete all items matching any pattern
for pattern in pattern_list:
    # Try to delete as feature classes
    for item in arcpy.ListFeatureClasses(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing feature class: {item}")
        except Exception as e:
            print(f"Warning: Could not delete feature class {item}. Error: {str(e)}")
    
    # Try to delete as tables
    for item in arcpy.ListTables(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing table: {item}")
        except Exception as e:
            print(f"Warning: Could not delete table {item}. Error: {str(e)}")

print("Cleanup complete - all previous outputs removed")

Deleted existing feature class: TEMP_Midpoints_553cced9
Cleanup complete - all previous outputs removed


In [187]:
# Create a unique name for the temporary midpoints
unique_id = str(uuid.uuid4())[:8]
midpoints_fc_name = f"TEMP_Midpoints_{unique_id}"
midpoints_fc_path = os.path.join(default_gdb, midpoints_fc_name)

# Create midpoints using the FeatureToPoint tool
print("\nGenerating midpoints using FeatureToPoint tool...")
if arcpy.Exists(midpoints_fc_path):
    arcpy.management.Delete(midpoints_fc_path)
    
arcpy.management.FeatureToPoint(
    PMP_Path, 
    midpoints_fc_path, 
    "CENTROID"  # Using CENTROID for lines will give the midpoint
)

print(f"Midpoints created successfully at: {midpoints_fc_path}")


Generating midpoints using FeatureToPoint tool...
Midpoints created successfully at: C:\Users\JLin\Downloads\QC Tool\Default.gdb\TEMP_Midpoints_67aeb36d


In [188]:
# Determine fields to retrieve from the original feature class
shapefile_retrieve_fields = ["SHAPE@"]

# If we have a combined field, use it
if street_sec_field:
    shapefile_retrieve_fields.append(street_sec_field)
# Otherwise use separate fields
else:
    shapefile_retrieve_fields.append(street_id_field)
    shapefile_retrieve_fields.append(section_id_field)
    
# Add additional fields if they exist
if street_name_field:
    shapefile_retrieve_fields.append(street_name_field)
if begin_loc_field:
    shapefile_retrieve_fields.append(begin_loc_field)
if end_loc_field:
    shapefile_retrieve_fields.append(end_loc_field)

# Join fields from original data with midpoint geometries
# First, create a dictionary of attributes from the original feature class
attributes_dict = {}

with arcpy.da.SearchCursor(wildomar_layer, shapefile_retrieve_fields) as cursor:
    for i, row in enumerate(cursor):
        # Get the shape
        shape = row[0]  # SHAPE@
        
        # Initialize with defaults
        street_id_value = ""
        section_id_value = ""
        street_name_value = ""
        begin_loc_value = ""
        end_loc_value = ""
        
        field_idx = 1  # Start from field after SHAPE@
        
        # Determine the key based on available fields
        if street_sec_field:
            street_sec_value = str(row[field_idx]).strip()
            key = street_sec_value
            street_id_value = street_sec_value
            section_id_value = street_sec_value
            field_idx += 1
        else:
            street_id_value = str(row[field_idx]).strip()
            section_id_value = str(row[field_idx + 1]).strip()
            key = f"{street_id_value} - {section_id_value}"
            field_idx += 2
        
        # Get additional fields if they exist
        if street_name_field:
            street_name_value = row[field_idx]
            field_idx += 1
        if begin_loc_field:
            begin_loc_value = row[field_idx]
            field_idx += 1
        if end_loc_field:
            end_loc_value = row[field_idx]
            field_idx += 1
        
        # Calculate midpoint
        midpoint = shape.positionAlongLine(0.5, True)
        
        # Project the point to WGS84 for API URLs
        proj = midpoint.projectAs(wgs84)
        lon = proj.firstPoint.X
        lat = proj.firstPoint.Y
        
        # Store original coordinates
        original_x = midpoint.firstPoint.X
        original_y = midpoint.firstPoint.Y
        
        # Store attributes
        attributes_dict[i] = {
            'street_id': street_id_value,
            'section_id': section_id_value,
            'combined_key': key,
            'street_name': street_name_value,
            'begin_loc': begin_loc_value,
            'end_loc': end_loc_value,
            'latitude': lat,
            'longitude': lon,
            'original_x': original_x,
            'original_y': original_y,
            'spatial_reference': layer_sr
        }

In [189]:
# ================================================================
# Cell 11 – Build output records with QC_Points field names
# ================================================================

print("\n🔎 Matching attributes with PCI data (all features)…")

results = []

def safe_float(v):
    try:
        return float(v)
    except (TypeError, ValueError):
        return None

for fid, attrs in attributes_dict.items():
    key = attrs["combined_key"]
    rec = {}                               # fresh dict per line feature
    rec.update(attrs)                      # copy original line attributes

    # ---- Pull PCI‑history values -----------------------------------------
    pci = pci_data.get(key)                # may be None
    
    # Debug output for key records
    if pci and pci.get('meets_criteria', False):
        print(f"DEBUG: Found matching criteria record for {key}")
        
    prev_date   = pci.get('prev_insp_date')      if pci else None
    prev_pci    = safe_float(pci.get('prev_pci')) if pci else None
    mr_date     = pci.get('m_r_date')           if pci else None
    mr_treat    = pci.get('m_r_treatment')      if pci else None
    last_date   = pci.get('last_insp_date')     if pci else None
    last_pci    = safe_float(pci.get('last_pci')) if pci else None
    pci_diff    = safe_float(pci.get('pci_diff_calc')) if pci else None

    rec["PrevInspDate"]    = prev_date
    rec["PrevInspPCI"]     = prev_pci
    rec["MRDate"]          = mr_date
    rec["MRTreatmentName"] = mr_treat
    rec["LastInspDate"]    = last_date
    rec["LastInspPCI"]     = last_pci
    rec["PCIDifference"]   = pci_diff

    # ---- Calculate diff & determine QC flag ------------------------------
    # Explicitly set QC flag based on meets_criteria
    if pci and pci.get('meets_criteria', False):
        rec["QC"] = "Y"
        print(f"Setting QC=Y for {key}: diff={pci_diff}")
    else:
        rec["QC"] = "N"

    results.append(rec)

print(f"✅ Created records for {len(results)} line features.")
print(f"   • {sum(r['QC']=='Y' for r in results)} meet the QC criteria.")


🔎 Matching attributes with PCI data (all features)…
DEBUG: Found matching criteria record for 1002 - 5466
Setting QC=Y for 1002 - 5466: diff=-14.0
DEBUG: Found matching criteria record for 1006 - 1287
Setting QC=Y for 1006 - 1287: diff=-18.0
DEBUG: Found matching criteria record for 1014 - 4483
Setting QC=Y for 1014 - 4483: diff=-17.0
DEBUG: Found matching criteria record for 1014 - 5947
Setting QC=Y for 1014 - 5947: diff=-15.0
DEBUG: Found matching criteria record for 1018 - 2222
Setting QC=Y for 1018 - 2222: diff=-17.0
DEBUG: Found matching criteria record for 1022 - 1139
Setting QC=Y for 1022 - 1139: diff=-16.0
DEBUG: Found matching criteria record for 1022 - 2727
Setting QC=Y for 1022 - 2727: diff=-14.0
DEBUG: Found matching criteria record for 1022 - 3070
Setting QC=Y for 1022 - 3070: diff=-13.0
DEBUG: Found matching criteria record for 1022 - 3096
Setting QC=Y for 1022 - 3096: diff=-13.0
DEBUG: Found matching criteria record for 1022 - 3097
Setting QC=Y for 1022 - 3097: diff=-13

Setting QC=Y for 1230 - 1152: diff=20.0
DEBUG: Found matching criteria record for 1242 - 2168
Setting QC=Y for 1242 - 2168: diff=-21.0
DEBUG: Found matching criteria record for 1242 - 5626
Setting QC=Y for 1242 - 5626: diff=-14.0
DEBUG: Found matching criteria record for 1252 - 5734
Setting QC=Y for 1252 - 5734: diff=-11.0
DEBUG: Found matching criteria record for 1256 - 5595
Setting QC=Y for 1256 - 5595: diff=-55.0
DEBUG: Found matching criteria record for 1266 - 5634
Setting QC=Y for 1266 - 5634: diff=-40.0
DEBUG: Found matching criteria record for 1266 - 5635
Setting QC=Y for 1266 - 5635: diff=-46.0
DEBUG: Found matching criteria record for 1266 - 7305
Setting QC=Y for 1266 - 7305: diff=-22.0
DEBUG: Found matching criteria record for 1274 - 1874
Setting QC=Y for 1274 - 1874: diff=-14.0
DEBUG: Found matching criteria record for 1274 - 2544
Setting QC=Y for 1274 - 2544: diff=-11.0
DEBUG: Found matching criteria record for 1294 - 2546
Setting QC=Y for 1294 - 2546: diff=-16.0
DEBUG: Fou

Setting QC=Y for 1660 - 1549: diff=-16.0
DEBUG: Found matching criteria record for 1662 - 2126
Setting QC=Y for 1662 - 2126: diff=-24.0
DEBUG: Found matching criteria record for 1662 - 6428
Setting QC=Y for 1662 - 6428: diff=-19.0
DEBUG: Found matching criteria record for 1688 - 3218
Setting QC=Y for 1688 - 3218: diff=27.0
DEBUG: Found matching criteria record for 1690 - 3210
Setting QC=Y for 1690 - 3210: diff=28.0
DEBUG: Found matching criteria record for 1690 - 3211
Setting QC=Y for 1690 - 3211: diff=23.0
DEBUG: Found matching criteria record for 1690 - 2437
Setting QC=Y for 1690 - 2437: diff=31.0
DEBUG: Found matching criteria record for 1690 - 1155
Setting QC=Y for 1690 - 1155: diff=19.0
DEBUG: Found matching criteria record for 1690 - 7363
Setting QC=Y for 1690 - 7363: diff=27.0
DEBUG: Found matching criteria record for 1694 - 1926
Setting QC=Y for 1694 - 1926: diff=-16.0
DEBUG: Found matching criteria record for 1696 - 1471
Setting QC=Y for 1696 - 1471: diff=-15.0
DEBUG: Found ma

Setting QC=Y for 2002 - 1183: diff=-13.0
DEBUG: Found matching criteria record for 2002 - 1276
Setting QC=Y for 2002 - 1276: diff=-10.0
DEBUG: Found matching criteria record for 2004 - 1924
Setting QC=Y for 2004 - 1924: diff=-11.0
DEBUG: Found matching criteria record for 2006 - 1137
Setting QC=Y for 2006 - 1137: diff=-32.0
DEBUG: Found matching criteria record for 2028 - 4353
Setting QC=Y for 2028 - 4353: diff=-16.0
DEBUG: Found matching criteria record for 2030 - 5217
Setting QC=Y for 2030 - 5217: diff=-11.0
DEBUG: Found matching criteria record for 2030 - 1292
Setting QC=Y for 2030 - 1292: diff=-13.0
DEBUG: Found matching criteria record for 2032 - 1026
Setting QC=Y for 2032 - 1026: diff=-13.0
DEBUG: Found matching criteria record for 2032 - 5003
Setting QC=Y for 2032 - 5003: diff=-12.0
DEBUG: Found matching criteria record for 2032 - 5497
Setting QC=Y for 2032 - 5497: diff=-12.0
DEBUG: Found matching criteria record for 2034 - 5000
Setting QC=Y for 2034 - 5000: diff=-38.0
DEBUG: Fo

Setting QC=Y for 2250 - 2785: diff=-14.0
DEBUG: Found matching criteria record for 2250 - 6946
Setting QC=Y for 2250 - 6946: diff=-13.0
DEBUG: Found matching criteria record for 2250 - 6947
Setting QC=Y for 2250 - 6947: diff=-12.0
DEBUG: Found matching criteria record for 2250 - 6948
Setting QC=Y for 2250 - 6948: diff=-12.0
DEBUG: Found matching criteria record for 2250 - 5451
Setting QC=Y for 2250 - 5451: diff=-16.0
DEBUG: Found matching criteria record for 2258 - 1518
Setting QC=Y for 2258 - 1518: diff=-52.0
DEBUG: Found matching criteria record for 2260 - 5522
Setting QC=Y for 2260 - 5522: diff=-17.0
DEBUG: Found matching criteria record for 2268 - 1252
Setting QC=Y for 2268 - 1252: diff=-45.0
DEBUG: Found matching criteria record for 2268 - 3730
Setting QC=Y for 2268 - 3730: diff=-60.0
DEBUG: Found matching criteria record for 2268 - 4499
Setting QC=Y for 2268 - 4499: diff=-35.0
DEBUG: Found matching criteria record for 2270 - 4767
Setting QC=Y for 2270 - 4767: diff=-17.0
DEBUG: Fo

DEBUG: Found matching criteria record for 2472 - 5397
Setting QC=Y for 2472 - 5397: diff=24.0
DEBUG: Found matching criteria record for 2472 - 6019
Setting QC=Y for 2472 - 6019: diff=28.0
DEBUG: Found matching criteria record for 2472 - 6020
Setting QC=Y for 2472 - 6020: diff=21.0
DEBUG: Found matching criteria record for 2474 - 4320
Setting QC=Y for 2474 - 4320: diff=39.0
DEBUG: Found matching criteria record for 2490 - 4080
Setting QC=Y for 2490 - 4080: diff=-45.0
DEBUG: Found matching criteria record for 2490 - 4081
Setting QC=Y for 2490 - 4081: diff=-48.0
DEBUG: Found matching criteria record for 2490 - 4082
Setting QC=Y for 2490 - 4082: diff=-42.0
DEBUG: Found matching criteria record for 2490 - 4074
Setting QC=Y for 2490 - 4074: diff=-51.0
DEBUG: Found matching criteria record for 2490 - 5293
Setting QC=Y for 2490 - 5293: diff=36.0
DEBUG: Found matching criteria record for 2496 - 7155
Setting QC=Y for 2496 - 7155: diff=-13.0
DEBUG: Found matching criteria record for 2498 - 6029
S

Setting QC=Y for 2802 - 4211: diff=27.0
DEBUG: Found matching criteria record for 2802 - 4212
Setting QC=Y for 2802 - 4212: diff=16.0
DEBUG: Found matching criteria record for 2802 - 4213
Setting QC=Y for 2802 - 4213: diff=25.0
DEBUG: Found matching criteria record for 2802 - 4214
Setting QC=Y for 2802 - 4214: diff=21.0
DEBUG: Found matching criteria record for 2802 - 5305
Setting QC=Y for 2802 - 5305: diff=20.0
DEBUG: Found matching criteria record for 2804 - 4125
Setting QC=Y for 2804 - 4125: diff=16.0
DEBUG: Found matching criteria record for 2806 - 5694
Setting QC=Y for 2806 - 5694: diff=-14.0
DEBUG: Found matching criteria record for 2806 - 5695
Setting QC=Y for 2806 - 5695: diff=-12.0
DEBUG: Found matching criteria record for 2806 - 4913
Setting QC=Y for 2806 - 4913: diff=-14.0
DEBUG: Found matching criteria record for 2808 - 1468
Setting QC=Y for 2808 - 1468: diff=22.0
DEBUG: Found matching criteria record for 2812 - 4215
Setting QC=Y for 2812 - 4215: diff=18.0
DEBUG: Found matc

DEBUG: Found matching criteria record for 2962 - 2976
Setting QC=Y for 2962 - 2976: diff=19.0
DEBUG: Found matching criteria record for 2972 - 3539
Setting QC=Y for 2972 - 3539: diff=29.0
DEBUG: Found matching criteria record for 2972 - 5032
Setting QC=Y for 2972 - 5032: diff=18.0
DEBUG: Found matching criteria record for 2972 - 7412
Setting QC=Y for 2972 - 7412: diff=42.0
DEBUG: Found matching criteria record for 2974 - 4545
Setting QC=Y for 2974 - 4545: diff=16.0
DEBUG: Found matching criteria record for 2978 - 3212
Setting QC=Y for 2978 - 3212: diff=15.0
DEBUG: Found matching criteria record for 2994 - 1886
Setting QC=Y for 2994 - 1886: diff=-17.0
DEBUG: Found matching criteria record for 3000 - 3401
Setting QC=Y for 3000 - 3401: diff=17.0
DEBUG: Found matching criteria record for 3000 - 5277
Setting QC=Y for 3000 - 5277: diff=25.0
DEBUG: Found matching criteria record for 3000 - 4040
Setting QC=Y for 3000 - 4040: diff=33.0
DEBUG: Found matching criteria record for 3000 - 4042
Setti

DEBUG: Found matching criteria record for 525 - 1016
Setting QC=Y for 525 - 1016: diff=-38.0
DEBUG: Found matching criteria record for 540 - 6185
Setting QC=Y for 540 - 6185: diff=-10.0
DEBUG: Found matching criteria record for 545 - 2310
Setting QC=Y for 545 - 2310: diff=-24.0
DEBUG: Found matching criteria record for 555 - 4856
Setting QC=Y for 555 - 4856: diff=-13.0
DEBUG: Found matching criteria record for 570 - 4606
Setting QC=Y for 570 - 4606: diff=22.0
DEBUG: Found matching criteria record for 135 - 4015
Setting QC=Y for 135 - 4015: diff=-10.0
DEBUG: Found matching criteria record for 250 - 7047
Setting QC=Y for 250 - 7047: diff=-35.0
DEBUG: Found matching criteria record for 335 - 6483
Setting QC=Y for 335 - 6483: diff=-17.0
DEBUG: Found matching criteria record for 335 - 6763
Setting QC=Y for 335 - 6763: diff=-14.0
DEBUG: Found matching criteria record for 360 - 6917
Setting QC=Y for 360 - 6917: diff=-11.0
DEBUG: Found matching criteria record for 375 - 5453
Setting QC=Y for 3

In [190]:
# 1. Create the table with specified columns
print(f"Creating table: {table_name}")
arcpy.management.CreateTable(default_gdb, table_name)
arcpy.management.AddField(table_path, "StreetID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "SectionID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "StreetName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "BeginLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "EndLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "PrevInspDate", "DATE")
arcpy.management.AddField(table_path, "PrevInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "MRDate", "DATE")
arcpy.management.AddField(table_path, "MRTreatmentName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "LastInspDate", "DATE")
arcpy.management.AddField(table_path, "LastInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "PCIDifference", "DOUBLE")  # Add PCI Difference field
arcpy.management.AddField(table_path, "Lat", "DOUBLE")
arcpy.management.AddField(table_path, "Long", "DOUBLE")
arcpy.management.AddField(table_path, "MapillaryLink", "TEXT", field_length=255)
arcpy.management.AddField(table_path, "GoogleImageLink", "TEXT", field_length=255)

# Insert the data into the table
print("Inserting data into the table...")
with arcpy.da.InsertCursor(table_path, [
        "StreetID", "SectionID", "StreetName", "BeginLocation", "EndLocation",
        "PrevInspDate", "PrevInspPCI", "MRDate", "MRTreatmentName", 
        "LastInspDate", "LastInspPCI", "PCIDifference", "Lat", "Long", 
        "MapillaryLink", "GoogleImageLink"
    ]) as ic:
    for r in results:
        lat = r['latitude']
        lon = r['longitude']
        
        # Simplified Mapillary link with no filters
        map_url = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        
        # Google Street View link
        g_url = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"
        
        # Get values with defaults for missing fields
        street_id = r.get('street_id', "")
        section_id = r.get('section_id', "")
        street_name = r.get('street_name', "")
        begin_loc = r.get('begin_loc', "")
        end_loc = r.get('end_loc', "")
        prev_insp_date = r.get('prev_insp_date', None)
        prev_insp_pci = r.get('prev_pci', None)
        m_r_date = r.get('m_r_date', None)
        m_r_treatment = r.get('m_r_treatment', "")
        last_insp_date = r.get('last_insp_date', None)
        last_insp_pci = r.get('last_pci', None)
        
        # Calculate PCI difference
        pci_difference = r.get('pci_diff_calc', None)
        
        # Debug output for begin and end locations
        print(f"Record {street_id}-{section_id}: BeginLoc={begin_loc}, EndLoc={end_loc}")
        
        ic.insertRow([
            street_id, section_id, street_name, begin_loc, end_loc,
            prev_insp_date, prev_insp_pci, m_r_date, m_r_treatment,
            last_insp_date, last_insp_pci, pci_difference, lat, lon,
            map_url, g_url
        ])

Inserting data into the table...
Record 1002 - 5466-1002 - 5466: BeginLoc=, EndLoc=
Record 1002 - 5467-1002 - 5467: BeginLoc=, EndLoc=
Record 1004 - 4057-1004 - 4057: BeginLoc=, EndLoc=
Record 1004 - 5279-1004 - 5279: BeginLoc=, EndLoc=
Record 1004 - 1438-1004 - 1438: BeginLoc=, EndLoc=
Record 1004 - 3424-1004 - 3424: BeginLoc=, EndLoc=
Record 1004 - 7494-1004 - 7494: BeginLoc=, EndLoc=
Record 1006 - 1287-1006 - 1287: BeginLoc=, EndLoc=
Record 1008 - 4045-1008 - 4045: BeginLoc=, EndLoc=
Record 1010 - 6775-1010 - 6775: BeginLoc=, EndLoc=
Record 1012 - 5159-1012 - 5159: BeginLoc=, EndLoc=
Record 1014 - 4483-1014 - 4483: BeginLoc=, EndLoc=
Record 1014 - 5947-1014 - 5947: BeginLoc=, EndLoc=
Record 1016 - 6075-1016 - 6075: BeginLoc=, EndLoc=
Record 1018 - 6573-1018 - 6573: BeginLoc=, EndLoc=
Record 1018 - 6574-1018 - 6574: BeginLoc=, EndLoc=
Record 1018 - 6575-1018 - 6575: BeginLoc=, EndLoc=
Record 1018 - 6813-1018 - 6813: BeginLoc=, EndLoc=
Record 1018 - 6597-1018 - 6597: BeginLoc=, EndLoc

Record 1144 - 3991-1144 - 3991: BeginLoc=, EndLoc=
Record 1144 - 7443-1144 - 7443: BeginLoc=, EndLoc=
Record 1146 - 5802-1146 - 5802: BeginLoc=, EndLoc=
Record 1146 - 6117-1146 - 6117: BeginLoc=, EndLoc=
Record 1146 - 5870-1146 - 5870: BeginLoc=, EndLoc=
Record 1146 - 7267-1146 - 7267: BeginLoc=, EndLoc=
Record 1146 - 7268-1146 - 7268: BeginLoc=, EndLoc=
Record 1148 - 1616-1148 - 1616: BeginLoc=, EndLoc=
Record 1150 - 5174-1150 - 5174: BeginLoc=, EndLoc=
Record 1152 - 1335-1152 - 1335: BeginLoc=, EndLoc=
Record 1154 - 1341-1154 - 1341: BeginLoc=, EndLoc=
Record 1156 - 2408-1156 - 2408: BeginLoc=, EndLoc=
Record 1156 - 2409-1156 - 2409: BeginLoc=, EndLoc=
Record 1158 - 3846-1158 - 3846: BeginLoc=, EndLoc=
Record 1158 - 5428-1158 - 5428: BeginLoc=, EndLoc=
Record 1160 - 4614-1160 - 4614: BeginLoc=, EndLoc=
Record 1160 - 4615-1160 - 4615: BeginLoc=, EndLoc=
Record 1160 - 4616-1160 - 4616: BeginLoc=, EndLoc=
Record 1160 - 4617-1160 - 4617: BeginLoc=, EndLoc=
Record 1162 - 3759-1162 - 3759:

Record 1300 - 7160-1300 - 7160: BeginLoc=, EndLoc=
Record 1300 - 2224-1300 - 2224: BeginLoc=, EndLoc=
Record 1300 - 2225-1300 - 2225: BeginLoc=, EndLoc=
Record 1302 - 1765-1302 - 1765: BeginLoc=, EndLoc=
Record 1304 - 2080-1304 - 2080: BeginLoc=, EndLoc=
Record 1304 - 6403-1304 - 6403: BeginLoc=, EndLoc=
Record 1304 - 6404-1304 - 6404: BeginLoc=, EndLoc=
Record 1306 - 5084-1306 - 5084: BeginLoc=, EndLoc=
Record 1306 - 5085-1306 - 5085: BeginLoc=, EndLoc=
Record 1306 - 3649-1306 - 3649: BeginLoc=, EndLoc=
Record 1306 - 3650-1306 - 3650: BeginLoc=, EndLoc=
Record 1306 - 3651-1306 - 3651: BeginLoc=, EndLoc=
Record 1308 - 2179-1308 - 2179: BeginLoc=, EndLoc=
Record 1308 - 6498-1308 - 6498: BeginLoc=, EndLoc=
Record 1310 - 1858-1310 - 1858: BeginLoc=, EndLoc=
Record 1310 - 1859-1310 - 1859: BeginLoc=, EndLoc=
Record 1310 - 1860-1310 - 1860: BeginLoc=, EndLoc=
Record 1310 - 1861-1310 - 1861: BeginLoc=, EndLoc=
Record 1310 - 1862-1310 - 1862: BeginLoc=, EndLoc=
Record 1310 - 1863-1310 - 1863:

Record 1468 - 5098-1468 - 5098: BeginLoc=, EndLoc=
Record 1468 - 3680-1468 - 3680: BeginLoc=, EndLoc=
Record 1470 - 3679-1470 - 3679: BeginLoc=, EndLoc=
Record 1472 - 3548-1472 - 3548: BeginLoc=, EndLoc=
Record 1472 - 7483-1472 - 7483: BeginLoc=, EndLoc=
Record 1474 - 1997-1474 - 1997: BeginLoc=, EndLoc=
Record 1474 - 5814-1474 - 5814: BeginLoc=, EndLoc=
Record 1474 - 6253-1474 - 6253: BeginLoc=, EndLoc=
Record 1474 - 6254-1474 - 6254: BeginLoc=, EndLoc=
Record 1474 - 6255-1474 - 6255: BeginLoc=, EndLoc=
Record 1474 - 6256-1474 - 6256: BeginLoc=, EndLoc=
Record 1474 - 6257-1474 - 6257: BeginLoc=, EndLoc=
Record 1474 - 5915-1474 - 5915: BeginLoc=, EndLoc=
Record 1476 - 6662-1476 - 6662: BeginLoc=, EndLoc=
Record 1476 - 1647-1476 - 1647: BeginLoc=, EndLoc=
Record 1476 - 7301-1476 - 7301: BeginLoc=, EndLoc=
Record 1478 - 6411-1478 - 6411: BeginLoc=, EndLoc=
Record 1478 - 6413-1478 - 6413: BeginLoc=, EndLoc=
Record 1480 - 6062-1480 - 6062: BeginLoc=, EndLoc=
Record 1482 - 7013-1482 - 7013:

Record 1626 - 2202-1626 - 2202: BeginLoc=, EndLoc=
Record 1628 - 1837-1628 - 1837: BeginLoc=, EndLoc=
Record 1628 - 1838-1628 - 1838: BeginLoc=, EndLoc=
Record 1628 - 1839-1628 - 1839: BeginLoc=, EndLoc=
Record 1630 - 1836-1630 - 1836: BeginLoc=, EndLoc=
Record 1630 - 6098-1630 - 6098: BeginLoc=, EndLoc=
Record 1632 - 5795-1632 - 5795: BeginLoc=, EndLoc=
Record 1634 - 2153-1634 - 2153: BeginLoc=, EndLoc=
Record 1636 - 2151-1636 - 2151: BeginLoc=, EndLoc=
Record 1636 - 2152-1636 - 2152: BeginLoc=, EndLoc=
Record 1636 - 2013-1636 - 2013: BeginLoc=, EndLoc=
Record 1636 - 6293-1636 - 6293: BeginLoc=, EndLoc=
Record 5714 - 7548-5714 - 7548: BeginLoc=, EndLoc=
Record 1638 - 6251-1638 - 6251: BeginLoc=, EndLoc=
Record 1640 - 7209-1640 - 7209: BeginLoc=, EndLoc=
Record 1642 - 1779-1642 - 1779: BeginLoc=, EndLoc=
Record 1644 - 2641-1644 - 2641: BeginLoc=, EndLoc=
Record 1646 - 1584-1646 - 1584: BeginLoc=, EndLoc=
Record 1648 - 1301-1648 - 1301: BeginLoc=, EndLoc=
Record 1648 - 1302-1648 - 1302:

...

Record 1916 - 4431-1916 - 4431: BeginLoc=, EndLoc=
Record 1918 - 4432-1918 - 4432: BeginLoc=, EndLoc=
Record 1920 - 4863-1920 - 4863: BeginLoc=, EndLoc=
Record 1922 - 2773-1922 - 2773: BeginLoc=, EndLoc=
Record 1922 - 2774-1922 - 2774: BeginLoc=, EndLoc=
Record 1924 - 4821-1924 - 4821: BeginLoc=, EndLoc=
Record 1926 - 3033-1926 - 3033: BeginLoc=, EndLoc=
Record 1926 - 3034-1926 - 3034: BeginLoc=, EndLoc=
Record 1926 - 1544-1926 - 1544: BeginLoc=, EndLoc=
Record 1928 - 2752-1928 - 2752: BeginLoc=, EndLoc=
Record 1930 - 1819-1930 - 1819: BeginLoc=, EndLoc=
Record 1932 - 3047-1932 - 3047: BeginLoc=, EndLoc=
Record 1934 - 1260-1934 - 1260: BeginLoc=, EndLoc=
Record 1936 - 5446-1936 - 5446: BeginLoc=, EndLoc=
Record 1936 - 7542-1936 - 7542: BeginLoc=, EndLoc=
Record 1938 - 3484-1938 - 3484: BeginLoc=, EndLoc=
Record 1940 - 1119-1940 - 1119: BeginLoc=, EndLoc=
Record 1940 - 5156-1940 - 5156: BeginLoc=, EndLoc=
Record 1940 - 4457-1940 - 4457: BeginLoc=, EndLoc=
Record 1940 - 3949-1940 - 3949:

Record 2048 - 2294-2048 - 2294: BeginLoc=, EndLoc=
Record 2048 - 2295-2048 - 2295: BeginLoc=, EndLoc=
Record 2048 - 3174-2048 - 3174: BeginLoc=, EndLoc=
Record 2048 - 7544-2048 - 7544: BeginLoc=, EndLoc=
Record 2048 - 7468-2048 - 7468: BeginLoc=, EndLoc=
Record 2050 - 1707-2050 - 1707: BeginLoc=, EndLoc=
Record 2050 - 6873-2050 - 6873: BeginLoc=, EndLoc=
Record 2052 - 1928-2052 - 1928: BeginLoc=, EndLoc=
Record 2054 - 6923-2054 - 6923: BeginLoc=, EndLoc=
Record 3000A - 1000-3000A - 1000: BeginLoc=, EndLoc=
Record 2056 - 3722-2056 - 3722: BeginLoc=, EndLoc=
Record 2056 - 3723-2056 - 3723: BeginLoc=, EndLoc=
Record 2056 - 4263-2056 - 4263: BeginLoc=, EndLoc=
Record 2056 - 4264-2056 - 4264: BeginLoc=, EndLoc=
Record 2056 - 6036-2056 - 6036: BeginLoc=, EndLoc=
Record 2058 - 4262-2058 - 4262: BeginLoc=, EndLoc=
Record 2060 - 5349-2060 - 5349: BeginLoc=, EndLoc=
Record 2062 - 7377-2062 - 7377: BeginLoc=, EndLoc=
Record 2064 - 1342-2064 - 1342: BeginLoc=, EndLoc=
Record 2064 - 2627-2064 - 262

Record 445 - 4160-445 - 4160: BeginLoc=, EndLoc=
Record 445 - 4162-445 - 4162: BeginLoc=, EndLoc=
Record 530 - 1022-530 - 1022: BeginLoc=, EndLoc=
Record 555 - 2760-555 - 2760: BeginLoc=, EndLoc=
Record 555 - 4830-555 - 4830: BeginLoc=, EndLoc=
Record 555 - 5536-555 - 5536: BeginLoc=, EndLoc=
Record 565 - 2480-565 - 2480: BeginLoc=, EndLoc=
Record 570 - 3738-570 - 3738: BeginLoc=, EndLoc=
Record 130 - 5165-130 - 5165: BeginLoc=, EndLoc=
Record 130 - 6655-130 - 6655: BeginLoc=, EndLoc=
Record 305 - 12708-305 - 12708: BeginLoc=, EndLoc=
Record 305 - 12709-305 - 12709: BeginLoc=, EndLoc=
Record 305 - 3580-305 - 3580: BeginLoc=, EndLoc=
Record 330 - 12701-330 - 12701: BeginLoc=, EndLoc=
Record 330 - 12702-330 - 12702: BeginLoc=, EndLoc=
Record 370 - 6841-370 - 6841: BeginLoc=, EndLoc=
Record 375 - 3476-375 - 3476: BeginLoc=, EndLoc=
Record 375 - 5255-375 - 5255: BeginLoc=, EndLoc=
Record 470 - 2832-470 - 2832: BeginLoc=, EndLoc=
Record 525 - 1015-525 - 1015: BeginLoc=, EndLoc=
Record 570 -

Record 335 - 6763-335 - 6763: BeginLoc=, EndLoc=
Record 350 - 5997-350 - 5997: BeginLoc=, EndLoc=
Record 350 - 6954-350 - 6954: BeginLoc=, EndLoc=
Record 360 - 6917-360 - 6917: BeginLoc=, EndLoc=
Record 375 - 4030-375 - 4030: BeginLoc=, EndLoc=
Record 375 - 5453-375 - 5453: BeginLoc=, EndLoc=
Record 410 - 6751-410 - 6751: BeginLoc=, EndLoc=
Record 420 - 6393-420 - 6393: BeginLoc=, EndLoc=
Record 420 - 6396-420 - 6396: BeginLoc=, EndLoc=
Record 420 - 6443-420 - 6443: BeginLoc=, EndLoc=
Record 425 - 6762-425 - 6762: BeginLoc=, EndLoc=
Record 495 - 4905-495 - 4905: BeginLoc=, EndLoc=
Record 520 - 5717-520 - 5717: BeginLoc=, EndLoc=
Record 550 - 4812-550 - 4812: BeginLoc=, EndLoc=
Record 555 - 4930-555 - 4930: BeginLoc=, EndLoc=
Record 555 - 5642-555 - 5642: BeginLoc=, EndLoc=
Record 555 - 5779-555 - 5779: BeginLoc=, EndLoc=
Record 570 - 4632-570 - 4632: BeginLoc=, EndLoc=
Record 110 - 4361-110 - 4361: BeginLoc=, EndLoc=
Record 110 - 4362-110 - 4362: BeginLoc=, EndLoc=
Record 110 - 5211-11

Record 445 - 4239-445 - 4239: BeginLoc=, EndLoc=
Record 445 - 5283-445 - 5283: BeginLoc=, EndLoc=
Record 470 - 3882-470 - 3882: BeginLoc=, EndLoc=
Record 495 - 5619-495 - 5619: BeginLoc=, EndLoc=
Record 500 - 6603-500 - 6603: BeginLoc=, EndLoc=
Record 515 - 6341-515 - 6341: BeginLoc=, EndLoc=
Record 525 - 1017-525 - 1017: BeginLoc=, EndLoc=
Record 530 - 1020-530 - 1020: BeginLoc=, EndLoc=
Record 565 - 1953-565 - 1953: BeginLoc=, EndLoc=
Record 585 - 7503-585 - 7503: BeginLoc=, EndLoc=
Record 110 - 3592-110 - 3592: BeginLoc=, EndLoc=
Record 110 - 4355-110 - 4355: BeginLoc=, EndLoc=
Record 110 - 4356-110 - 4356: BeginLoc=, EndLoc=
Record 110 - 5011-110 - 5011: BeginLoc=, EndLoc=
Record 110 - 5526-110 - 5526: BeginLoc=, EndLoc=
Record 125 - 5909-125 - 5909: BeginLoc=, EndLoc=
Record 130 - 3049-130 - 3049: BeginLoc=, EndLoc=
Record 145 - 2651-145 - 2651: BeginLoc=, EndLoc=
Record 145 - 4608-145 - 4608: BeginLoc=, EndLoc=
Record 145 - 5282-145 - 5282: BeginLoc=, EndLoc=
Record 150 - 5373-15

In [191]:
# ================================================================
# Cell 13 – Create & populate QC_Points feature class
# ================================================================

print(f"\nCreating points feature class in WGS84: {fc_name}")
arcpy.management.CreateFeatureclass(
    default_gdb, fc_name, "POINT", spatial_reference=wgs84
)

# ---- Add fields -------------------------------------------------
field_defs = [
    ("StreetID",          "TEXT",   50),
    ("SectionID",         "TEXT",   50),
    ("StreetName",        "TEXT",  100),
    ("BeginLocation",     "TEXT",  100),
    ("EndLocation",       "TEXT",  100),
    ("PrevInspDate",      "DATE"),
    ("PrevInspPCI",       "DOUBLE"),
    ("MRDate",            "DATE"),
    ("MRTreatmentName",   "TEXT",  100),
    ("LastInspDate",      "DATE"),
    ("LastInspPCI",       "DOUBLE"),
    ("PCIDifference",     "DOUBLE"),
    ("Lat",               "DOUBLE"),
    ("Long",              "DOUBLE"),
    ("MapillaryLink",     "TEXT", 255),
    ("GoogleImageLink",   "TEXT", 255),
    ("QC",                "TEXT",   1)   # <‑‑ NEW
]
for name, ftype, *rest in field_defs:
    length = rest[0] if rest else None
    arcpy.management.AddField(fc_path, name, ftype, field_length=length)

# ---- Insert rows ------------------------------------------------
print("Populating the points feature class…")
with arcpy.da.InsertCursor(
        fc_path,
        ["SHAPE@", "StreetID", "SectionID", "StreetName", "BeginLocation",
         "EndLocation", "PrevInspDate", "PrevInspPCI", "MRDate",
         "MRTreatmentName", "LastInspDate", "LastInspPCI",
         "PCIDifference", "Lat", "Long", "MapillaryLink",
         "GoogleImageLink", "QC"]        # <‑‑ QC added
) as curs:
    for r in results:
        pt   = arcpy.Point(r["longitude"], r["latitude"])
        geom = arcpy.PointGeometry(pt, wgs84)

        lat, lon  = r["latitude"], r["longitude"]
        map_link  = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        gsv_link  = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"

        curs.insertRow([
            geom,
            r.get("street_id"), r.get("section_id"), r.get("street_name"),
            r.get("begin_loc"), r.get("end_loc"),
            r.get("PrevInspDate"), r.get("PrevInspPCI"), r.get("MRDate"),
            r.get("MRTreatmentName"), r.get("LastInspDate"),
            r.get("LastInspPCI"), r.get("PCIDifference"),
            lat, lon, map_link, gsv_link, r["QC"]     # <‑‑ QC value
        ])

print("✅ Points created & attributes populated.")



Creating points feature class in WGS84: QC_Points
Populating the points feature class…
✅ Points created & attributes populated.


In [192]:
# Define the add_layer_and_style function first
def add_layer_and_style(layer_path, layer_type="POINT", color=None, size=None, use_tear_pin=False):
    """
    Add a layer to the map only if it doesn't already exist, and apply styling.
    Added support for Tear Pin symbology for points.
    Returns the layer if it was added, None otherwise.
    """
    print(f"Adding layer: {os.path.basename(layer_path)}")
    
    # Check if layer already exists in the map
    existing_layer = None
    for lyr in active_map.listLayers():
        try:
            if lyr.dataSource == layer_path:
                existing_layer = lyr
                print(f"Layer already exists in map: {lyr.name}")
                break
        except:
            continue
    
    # If layer doesn't exist, add it
    if not existing_layer:
        active_map.addDataFromPath(layer_path)
        
        # Find the newly added layer
        for lyr in active_map.listLayers():
            try:
                if lyr.dataSource == layer_path:
                    existing_layer = lyr
                    print(f"Added layer to map: {lyr.name}")
                    break
            except:
                continue
    
    # Style the layer if found
    if existing_layer:
        if layer_type == "POINT" and use_tear_pin:
            try:
                # Attempt to set the Tear Pin symbol
                print("Attempting to apply Tear Pin symbol...")
                
                # Method 1: Try using the symbology classes
                try:
                    symb = existing_layer.symbology
                    
                    # Step 1: Get the current symbol
                    if hasattr(symb, 'renderer') and hasattr(symb.renderer, 'symbol'):
                        # Try to change the symbol style
                        try:
                            # Change the symbol type to a marker symbol
                            symb.renderer.symbol.style = "Tear Pin 1"
                            
                            # Set the size if provided
                            if size:
                                symb.renderer.symbol.size = size
                            else:
                                symb.renderer.symbol.size = 12  # Default tear pin size
                                
                            # Set the color if provided
                            if color:
                                symb.renderer.symbol.color = {'RGB': color}
                                
                            # Apply the symbology
                            existing_layer.symbology = symb
                            print("Applied Tear Pin symbol to point layer")
                        except Exception as e:
                            print(f"Could not set Tear Pin style: {str(e)}")
                            
                            # Try alternative method - just set size and color
                            try:
                                if size:
                                    symb.renderer.symbol.size = size
                                if color:
                                    symb.renderer.symbol.color = {'RGB': color}
                                existing_layer.symbology = symb
                                print("Applied basic styling (size/color) to point layer")
                                print("Please manually change the symbol to Tear Pin 1:")
                                print("1. Right-click the layer → Symbology")
                                print("2. Click on the symbol")
                                print("3. In the Format Point Symbol pane, select 'Tear Pin 1' from the gallery")
                            except:
                                print("Could not apply even basic styling")
                except Exception as e:
                    print(f"Could not modify symbol: {str(e)}")
                    print("Please manually set the Tear Pin symbol:")
                    print("1. Right-click the point layer in the Contents pane")
                    print("2. Select 'Symbology'")
                    print("3. Click on the current symbol to open the symbol selector")
                    print("4. Choose 'Tear Pin 1' from the gallery")
                    print("5. Set the appropriate size and color")
            except Exception as e:
                print(f"Could not apply Tear Pin styling: {str(e)}")
        elif color:  # Regular styling for other layer types or when not using Tear Pin
            try:
                symb = existing_layer.symbology
                if hasattr(symb, 'renderer'):
                    if hasattr(symb.renderer, 'symbol'):
                        if color:
                            symb.renderer.symbol.color = {'RGB': color}
                        if size and layer_type == "POINT":
                            symb.renderer.symbol.size = size
                        existing_layer.symbology = symb
                        print(f"Applied styling to layer: {existing_layer.name}")
            except Exception as e:
                print(f"Could not apply styling: {str(e)}")
    
    return existing_layer

# Now use the function to add layers only if needed, with proper styling
print("\nAdding outputs to the map (avoiding duplicates)...")

# Add point features with red styling
point_layer = add_layer_and_style(fc_path, "POINT", [255, 0, 0, 100], 8)

# Add table (tables don't need styling)
print("Adding table to map...")
existing_table = None
for tbl in active_map.listTables():
    try:
        if tbl.dataSource == table_path:
            existing_table = tbl
            print(f"Table already exists in map: {tbl.name}")
            break
    except:
        continue

if not existing_table:
    active_map.addDataFromPath(table_path)
    print(f"Added table to map: {table_name}")

# Save the project to preserve changes
aprx.save()
print("Project saved successfully with all changes.")
print("\nScript completed successfully!")
print(f"Created table: {table_name}")
print(f"Created feature class: {fc_name}")
print(f"Found {len(results)} points that met the threshold criteria.")

# Summary report of Begin/End Location fields processing
print("\n===== Begin/End Location Fields Summary =====")
begin_loc_feature_found = "Yes" if begin_loc_field else "No"
end_loc_feature_found = "Yes" if end_loc_field else "No"
begin_loc_pci_found = "Yes" if begin_loc_field_pci else "No"
end_loc_pci_found = "Yes" if end_loc_field_pci else "No"

print(f"Begin Location field found in feature layer: {begin_loc_feature_found}")
print(f"End Location field found in feature layer: {end_loc_feature_found}")
print(f"Begin Location field found in PCI table: {begin_loc_pci_found}")
print(f"End Location field found in PCI table: {end_loc_pci_found}")
print("============================================")


Adding outputs to the map (avoiding duplicates)...
Adding layer: QC_Points
Layer already exists in map: QC_Points
Applied styling to layer: QC_Points
Adding table to map...
Table already exists in map: QC_Table
Project saved successfully with all changes.

Script completed successfully!
Created table: QC_Table
Created feature class: QC_Points
Found 6334 points that met the threshold criteria.

===== Begin/End Location Fields Summary =====
Begin Location field found in feature layer: No
End Location field found in feature layer: No
Begin Location field found in PCI table: Yes
End Location field found in PCI table: Yes


In [193]:
# This cell is optional - you can create a wrapper function to call everything
def find_midpoints_with_pci_matching():
    """Main function to execute the workflow"""
    try:
        # All the code from cells 3-15 would go here
        print("Completed successfully!")
    except Exception as e:
        import traceback
        print(f"ERROR: An exception occurred during script execution:\n{str(e)}")
        print(traceback.format_exc())
        
        # Try to save any incomplete but important data
        try:
            if 'aprx' in locals():
                aprx.save()
                print("Project saved despite error.")
        except:
            print("Could not save project after error.")

# Uncomment the line below if you want to run everything at once
# find_midpoints_with_pci_matching()

In [194]:
# ---------------------------------------------------------------
# Spatial‑join back to the original line feature with QC field at the end
# ---------------------------------------------------------------
print("\n===== Performing Spatial Join to Original Line Feature =====")

# Verify that the paths exist before proceeding
print("Verifying paths...")
print(f"PMP_Path: {PMP_Path}")
print(f"fc_path: {fc_path}")

if not arcpy.Exists(PMP_Path):
    print(f"ERROR: Target feature class does not exist at {PMP_Path}")
    # Try to find the feature class in the current project
    print("Attempting to locate the feature class in the current project...")
    
    # Get the active map and find the feature layer again
    PMP_Path = None
    active_map = None
    
    for m in aprx.listMaps():
        for lyr in m.listLayers():
            if lyr.isFeatureLayer and "SSstreet" in lyr.name:
                PMP_Path = lyr.dataSource
                wildomar_layer = lyr
                active_map = m
                print(f"Found layer: {lyr.name} in map: {m.name}")
                print(f"Using data source: {PMP_Path}")
                break
        if PMP_Path:
            break
    
    if not PMP_Path:
        print("ERROR: Could not find the feature class in the project.")
        print("Unable to proceed with spatial join.")
        # Exit cell execution gracefully
        raise ValueError("Target feature class not found")

if not arcpy.Exists(fc_path):
    print(f"ERROR: Points feature class does not exist at {fc_path}")
    print("Unable to proceed with spatial join.")
    # Exit cell execution gracefully
    raise ValueError("Points feature class not found")

# 1) Field mapping ------------------------------------------------
print("Setting up field mapping...")

# First, get all fields from the target feature class except QC
try:
    target_fields = [f.name for f in arcpy.ListFields(PMP_Path) if f.name.upper() != "QC" and not f.name.upper().startswith("QC_")]
    print(f"Found {len(target_fields)} fields in target feature class")
except Exception as e:
    print(f"ERROR listing fields from target: {str(e)}")
    print("Using layer object directly instead of path...")
    # Try using the layer object directly
    target_fields = [f.name for f in arcpy.ListFields(wildomar_layer) if f.name.upper() != "QC" and not f.name.upper().startswith("QC_")]
    print(f"Found {len(target_fields)} fields in target layer")

# Create a FieldMappings object for the spatial join
field_mapping = arcpy.FieldMappings()

# Add all fields from the target feature class first (except QC)
for field_name in target_fields:
    if field_name not in ["OBJECTID", "SHAPE", "Shape", "SHAPE_Length", "Shape_Length"]:
        try:
            # Create a field map for each field
            field_map = arcpy.FieldMap()
            
            # Try using PMP_Path first, fall back to wildomar_layer if needed
            try:
                field_map.addInputField(PMP_Path, field_name)
            except:
                field_map.addInputField(wildomar_layer, field_name)
                
            # Add the field map to the mappings
            field_mapping.addFieldMap(field_map)
        except Exception as e:
            print(f"Warning: Could not add field {field_name}: {str(e)}")

# Now add selected fields from the points feature class
selected_fields = [
    {"name": "PrevInspDate",    "alias": "Previous_Date"},
    {"name": "PrevInspPCI",     "alias": "Previous_PCI"},
    {"name": "MRDate",          "alias": "MR_Date"},
    {"name": "MRTreatmentName", "alias": "MR_Treatment"},
    {"name": "LastInspDate",    "alias": "Last_Insp_Date"},
    {"name": "LastInspPCI",     "alias": "Last_Insp_PCI"},
    {"name": "PCIDifference",   "alias": "PCI_Diff"},
    {"name": "MapillaryLink",   "alias": "Mapillary"},
    {"name": "GoogleImageLink", "alias": "Google"},
]

# Add each field from the points to the mapping
for fld in selected_fields:
    try:
        # Create a field map for this field
        fm = arcpy.FieldMap()
        
        # Add the input field from the points feature class
        fm.addInputField(fc_path, fld["name"])
        
        # Set the output field properties
        out_fld = fm.outputField
        out_fld.name = fld["alias"]
        out_fld.aliasName = fld["alias"]
        fm.outputField = out_fld
        
        # Add the field map to the mappings
        field_mapping.addFieldMap(fm)
    except Exception as e:
        print(f"Warning: Could not add field {fld['name']}: {str(e)}")

# Finally, add the QC field LAST to ensure it appears at the end
try:
    qc_field_map = arcpy.FieldMap()
    qc_field_map.addInputField(fc_path, "QC")
    out_qc_field = qc_field_map.outputField
    out_qc_field.name = "QC"
    out_qc_field.aliasName = "QC"
    qc_field_map.outputField = out_qc_field
    field_mapping.addFieldMap(qc_field_map)
    print(f"Added QC field as the last field")
except Exception as e:
    print(f"Warning: Could not add QC field: {str(e)}")

print(f"Field mapping set up with QC field positioned last")

# 2) Run the spatial join ---------------------------------------
print(f"Creating joined feature class: {joined_fc_name}")

# First delete the output if it already exists
if arcpy.Exists(joined_fc_path):
    arcpy.management.Delete(joined_fc_path)
    print(f"Deleted existing {joined_fc_name}")

# Execute the spatial join - try with direct layer objects if paths fail
try:
    # Try using the paths
    arcpy.analysis.SpatialJoin(
        target_features=PMP_Path,
        join_features=fc_path,
        out_feature_class=joined_fc_path,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_ALL",
        field_mapping=field_mapping,
        match_option="CLOSEST",
        search_radius="10 Meters"
    )
    print("Spatial join completed using paths")
except Exception as e:
    print(f"Error using paths for spatial join: {str(e)}")
    print("Trying with layer objects...")
    
    try:
        # Try using the layer objects directly
        arcpy.analysis.SpatialJoin(
            target_features=wildomar_layer,
            join_features=active_map.listLayers(fc_name)[0] if fc_name in [lyr.name for lyr in active_map.listLayers()] else fc_path,
            out_feature_class=joined_fc_path,
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            field_mapping=field_mapping,
            match_option="CLOSEST",
            search_radius="10 Meters"
        )
        print("Spatial join completed using layer objects")
    except Exception as e2:
        print(f"Error with layer objects as well: {str(e2)}")
        print("Unable to complete spatial join")
        raise e2

# 3) Verify QC field is populated correctly ---------------------
if arcpy.Exists(joined_fc_path):
    print("Verifying QC field in joined shapefile...")

    # First, check if QC or QC_1 field exists
    field_names = [f.name for f in arcpy.ListFields(joined_fc_path)]
    has_qc = "QC" in field_names
    has_qc_1 = "QC_1" in field_names

    print(f"Fields in joined shapefile:")
    print(f"QC field exists: {has_qc}")
    print(f"QC_1 field exists: {has_qc_1}")

    # Determine which QC field to use
    use_qc_field = "QC" if has_qc else ("QC_1" if has_qc_1 else None)

    if use_qc_field:
        print(f"Using {use_qc_field} field. Checking values...")
        
        # Check QC field values
        y_count = 0
        n_count = 0
        null_count = 0
        
        with arcpy.da.SearchCursor(joined_fc_path, [use_qc_field]) as cursor:
            for row in cursor:
                if row[0] == "Y":
                    y_count += 1
                elif row[0] == "N":
                    n_count += 1
                else:
                    null_count += 1
        
        print(f"QC field values: Y={y_count}, N={n_count}, Null={null_count}")
        
        # If QC values aren't populated correctly, update them
        if y_count == 0 or null_count > 0:
            print("QC field not populated correctly. Running update...")
            
            # First, set all to "N"
            with arcpy.da.UpdateCursor(joined_fc_path, [use_qc_field]) as cursor:
                for row in cursor:
                    row[0] = "N"
                    cursor.updateRow(row)
            
            # Then set appropriate records to "Y"
            y_keys = []
            with arcpy.da.SearchCursor(fc_path, ["StreetID", "SectionID", "QC"]) as cursor:
                for row in cursor:
                    if row[2] == "Y":
                        street_id = str(row[0]).strip() if row[0] else ""
                        section_id = str(row[1]).strip() if row[1] else ""
                        key = f"{street_id} - {section_id}"
                        y_keys.append(key)
            
            print(f"Found {len(y_keys)} records with QC=Y in points layer")
            
            updated_count = 0
            with arcpy.da.UpdateCursor(joined_fc_path, ["StreetID", "SectionID", use_qc_field]) as cursor:
                for row in cursor:
                    street_id = str(row[0]).strip() if row[0] else ""
                    section_id = str(row[1]).strip() if row[1] else ""
                    key = f"{street_id} - {section_id}"
                    
                    if key in y_keys:
                        row[2] = "Y"
                        cursor.updateRow(row)
                        updated_count += 1
            
            print(f"Updated {updated_count} records to {use_qc_field}=Y")
    else:
        print("ERROR: No QC field found in joined shapefile!")
else:
    print("ERROR: Joined shapefile was not created successfully")

print("✅ Spatial join process completed")


===== Performing Spatial Join to Original Line Feature =====
Verifying paths...
PMP_Path: C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
fc_path: C:\Users\JLin\Downloads\QC Tool\Default.gdb\QC_Points
ERROR: Target feature class does not exist at C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
Attempting to locate the feature class in the current project...
Found layer: SSstreet in map: Map
Using data source: C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet
Setting up field mapping...
ERROR listing fields from target: "C:\Users\JLin\Documents\ArcGIS\Packages\QC Tool_e987fe\commondata\ssstreet_4\SSstreet" does not exist
Using layer object directly instead of path...
Found 28 fields in target layer
Added QC field as the last field
Field mapping set up with QC field positioned last
Creating joined feature class: QC_Joined_Shapefile
Deleted existing QC_Joined_Shapefile
Spatial 

In [195]:
# ---------------------------------------------------------------
# Fix for Duplicate QC Fields in QC_Joined_Shapefile
# ---------------------------------------------------------------
print("\n===== FIXING DUPLICATE QC FIELDS =====")

# Check if the joined feature class exists
if not arcpy.Exists(joined_fc_path):
    print(f"ERROR: {joined_fc_path} does not exist!")
else:
    # Check which QC fields exist
    field_names = [f.name for f in arcpy.ListFields(joined_fc_path)]
    has_qc = "QC" in field_names
    has_qc_1 = "QC_1" in field_names
    
    print(f"Fields in joined shapefile:")
    print(f"QC field exists: {has_qc}")
    print(f"QC_1 field exists: {has_qc_1}")
    
    # If both exist, we need to decide which one to use
    if has_qc and has_qc_1:
        print("Both QC and QC_1 fields exist. Examining values...")
        
        # Check if either field has values
        qc_has_values = False
        qc_1_has_values = False
        
        with arcpy.da.SearchCursor(joined_fc_path, ["QC"]) as cursor:
            for row in cursor:
                if row[0] is not None and row[0] != "":
                    qc_has_values = True
                    break
        
        with arcpy.da.SearchCursor(joined_fc_path, ["QC_1"]) as cursor:
            for row in cursor:
                if row[0] is not None and row[0] != "":
                    qc_1_has_values = True
                    break
        
        print(f"QC field has values: {qc_has_values}")
        print(f"QC_1 field has values: {qc_1_has_values}")
        
        # Decide which field to use
        use_qc_field = "QC"
        if qc_1_has_values and not qc_has_values:
            use_qc_field = "QC_1"
            print(f"Using {use_qc_field} field as it has values")
        else:
            print(f"Using {use_qc_field} field")
    
    elif has_qc:
        use_qc_field = "QC"
        print(f"Using {use_qc_field} field")
    elif has_qc_1:
        use_qc_field = "QC_1"
        print(f"Using {use_qc_field} field")
    else:
        # Add the QC field if neither exists
        print("No QC field found. Adding QC field...")
        arcpy.management.AddField(joined_fc_path, "QC", "TEXT", field_length=1)
        use_qc_field = "QC"
    
    # First, set all records to "N" in the selected QC field
    print(f"Setting all records to 'N' in {use_qc_field} field...")
    with arcpy.da.UpdateCursor(joined_fc_path, [use_qc_field]) as cursor:
        for row in cursor:
            row[0] = "N"
            cursor.updateRow(row)
    
    # Create a list of keys (Street_ID - Section_ID) that should be "Y"
    y_keys = []
    with arcpy.da.SearchCursor(fc_path, ["StreetID", "SectionID", "QC"]) as cursor:
        for row in cursor:
            if row[2] == "Y":
                street_id = str(row[0]).strip() if row[0] else ""
                section_id = str(row[1]).strip() if row[1] else ""
                key = f"{street_id} - {section_id}"
                y_keys.append(key)
    
    print(f"Found {len(y_keys)} records with QC=Y in points layer")
    
    # Now update the matching records in the joined feature class
    updated_count = 0
    
    with arcpy.da.UpdateCursor(joined_fc_path, ["StreetID", "SectionID", use_qc_field]) as cursor:
        for row in cursor:
            street_id = str(row[0]).strip() if row[0] else ""
            section_id = str(row[1]).strip() if row[1] else ""
            key = f"{street_id} - {section_id}"
            
            if key in y_keys:
                row[2] = "Y"
                cursor.updateRow(row)
                updated_count += 1
    
    print(f"Updated {updated_count} records to {use_qc_field}=Y using cursor")
    
    # Final verification
    y_count = 0
    n_count = 0
    null_count = 0
    
    with arcpy.da.SearchCursor(joined_fc_path, [use_qc_field]) as cursor:
        for row in cursor:
            if row[0] == "Y":
                y_count += 1
            elif row[0] == "N":
                n_count += 1
            else:
                null_count += 1
    
    print(f"\nFinal check of {use_qc_field} field in {joined_fc_name}:")
    print(f"Y count: {y_count}")
    print(f"N count: {n_count}")
    print(f"Null count: {null_count}")
    
    # Optional: Drop the unused QC field if both exist
    if has_qc and has_qc_1 and use_qc_field == "QC":
        print("Dropping unused QC_1 field...")
        arcpy.management.DeleteField(joined_fc_path, "QC_1")
    elif has_qc and has_qc_1 and use_qc_field == "QC_1":
        print("Dropping unused QC field...")
        arcpy.management.DeleteField(joined_fc_path, "QC")
    
    print("===== QC FIELD FIX COMPLETED =====")


===== FIXING DUPLICATE QC FIELDS =====
Fields in joined shapefile:
QC field exists: True
QC_1 field exists: False
Using QC field
Setting all records to 'N' in QC field...
Found 1737 records with QC=Y in points layer
Updated 0 records to QC=Y using cursor

Final check of QC field in QC_Joined_Shapefile:
Y count: 0
N count: 6334
Null count: 0
===== QC FIELD FIX COMPLETED =====
